# Check for germplasm mistakes
Gabriel Besombes   
23/05/2022     
# <br>    

## Context
----

Sometimes the wrong concepts from agrovoc are used to create germplasms in opensilex. The most common mistacke is to use a produce instead of the species that produces it. For example using "wheat" (the grain) instead of "Triticum" (the species).   
This notebook aims at exploring these mistakes automatically and exporting a list of potential mistakes as a csv file.   
This is perfectly safe as it __does not modify anything on opensilex.__    
A service exists in opensilex to properly change the URIs but this __should only be attempted if you are sure of what you are doing!__

---
# <br>

## Imports
--- 
These dependencies are needed :
* `pandas` is used to manipulate the data.    
* `SPARQLWrapper` is used to interract with a sparql endpoint.    

In [1]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

---    
# <br>    

## Check for general mistakes with agrovoc germplasms
---

Get the germplasms from opensilex that have agrovoc URIs

In [2]:
sparql = SPARQLWrapper(
    "http://127.0.0.1:8080/rdf4j-server/repositories/diaphen"
)
sparql.setReturnFormat(JSON)

In [3]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  vocabulary: <http://www.opensilex.org/vocabulary/oeso#>
SELECT ?s
WHERE {
    ?s rdf:type ?type .
    ?type (rdfs:subClassOf)* vocabulary:Germplasm .
    FILTER regex(str(?s), "http://aims.fao.org/aos/agrovoc", "i")
}
LIMIT 1000
"""

In [4]:
sparql.setQuery(query)
ret = sparql.queryAndConvert()
agrovoc_germplasms = [row["s"]["value"] for row in ret["results"]["bindings"]]
agrovoc_germplasms

['http://aims.fao.org/aos/agrovoc/c_12332',
 'http://aims.fao.org/aos/agrovoc/c_15476',
 'http://aims.fao.org/aos/agrovoc/c_3662',
 'http://aims.fao.org/aos/agrovoc/c_14386',
 'http://aims.fao.org/aos/agrovoc/c_6116',
 'http://aims.fao.org/aos/agrovoc/c_1066',
 'http://aims.fao.org/aos/agrovoc/c_13199',
 'http://aims.fao.org/aos/agrovoc/c_7247',
 'http://aims.fao.org/aos/agrovoc/c_8283',
 'http://aims.fao.org/aos/agrovoc/c_3339',
 'http://aims.fao.org/aos/agrovoc/c_5438',
 'http://aims.fao.org/aos/agrovoc/c_4555',
 'http://aims.fao.org/aos/agrovoc/c_29128',
 'http://aims.fao.org/aos/agrovoc/c_11456',
 'http://aims.fao.org/aos/agrovoc/c_7499',
 'http://aims.fao.org/aos/agrovoc/c_3558',
 'http://aims.fao.org/aos/agrovoc/c_14477',
 'http://aims.fao.org/aos/agrovoc/c_5339',
 'http://aims.fao.org/aos/agrovoc/c_3539',
 'http://aims.fao.org/aos/agrovoc/c_4464',
 'http://aims.fao.org/aos/agrovoc/c_5933',
 'http://aims.fao.org/aos/agrovoc/c_8220',
 'http://aims.fao.org/aos/agrovoc/c_6455',
 'ht

Check if these germplasms are part of the `organisms` concept in agrovoc. The ones that aren't are probably mistakes.

In [5]:
sparql = SPARQLWrapper(
    "https://agrovoc.fao.org/sparql"
)
sparql.setReturnFormat(JSON)

In [6]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?uri
WHERE {
    FILTER NOT EXISTS{
        <http://aims.fao.org/aos/agrovoc/c_49904>  <http://www.w3.org/2004/02/skos/core#narrower>* ?uri .
    }
    
    VALUES ?uri {%s}
}
LIMIT 1000
"""%("<" + "> <".join(agrovoc_germplasms) + ">")

In [7]:
sparql.setQuery(query)
ret = sparql.queryAndConvert()
agrovoc_wrong_germplasms = [row["uri"]["value"] for row in ret["results"]["bindings"]]
agrovoc_wrong_germplasms

['http://aims.fao.org/aos/agrovoc/c_12332',
 'http://aims.fao.org/aos/agrovoc/c_14386',
 'http://aims.fao.org/aos/agrovoc/c_11456',
 'http://aims.fao.org/aos/agrovoc/c_7499',
 'http://aims.fao.org/aos/agrovoc/c_3558',
 'http://aims.fao.org/aos/agrovoc/c_14477']

In [8]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?uri ?label
WHERE {
    ?uri <http://www.w3.org/2004/02/skos/core#prefLabel> ?label .
    FILTER (lang(?label) = "en")
    
    VALUES ?uri {%s}
}
LIMIT 1000
"""%("<" + "> <".join(agrovoc_wrong_germplasms) + ">")

In [9]:
sparql.setQuery(query)
ret = sparql.queryAndConvert()
tmp_d = {"uri" : [], "en_label" : []}
for row in ret["results"]["bindings"]:
    tmp_d["uri"].append(row["uri"]["value"])
    tmp_d["en_label"].append(row["label"]["value"])
agrovoc_wrong_germplasms = pd.DataFrame(tmp_d)
agrovoc_wrong_germplasms

,uri,en_label
0,http://aims.fao.org/aos/agrovoc/c_12332,maize
1,http://aims.fao.org/aos/agrovoc/c_14386,soft wheat
2,http://aims.fao.org/aos/agrovoc/c_11456,hard wheat
3,http://aims.fao.org/aos/agrovoc/c_7499,sugar beet
4,http://aims.fao.org/aos/agrovoc/c_3558,hemp
5,http://aims.fao.org/aos/agrovoc/c_14477,soybeans


---      

# <br>        

## Isolate produces
---

Some of these germplasms are produces/substances instead of the species/variety of the plant that produces it.     
Ex : Wheat (the grain) instead of Triticum (the plant species).

In [10]:
sparql = SPARQLWrapper(
    "https://agrovoc.fao.org/sparql"
)
sparql.setReturnFormat(JSON)

In [11]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?product_uri ?product_label ?produced_by_uri ?produced_by_label
WHERE {
    {<http://aims.fao.org/aos/agrovoc/c_8171>  <http://www.w3.org/2004/02/skos/core#narrower>* ?product_uri}
    UNION {<http://aims.fao.org/aos/agrovoc/c_330705>  <http://www.w3.org/2004/02/skos/core#narrower>* ?product_uri}
    ?product_uri <http://aims.fao.org/aos/agrontology#isProducedBy> ?produced_by_uri .

    ?product_uri <http://www.w3.org/2004/02/skos/core#prefLabel> ?product_label .
    FILTER (lang(?product_label) = "en")

    ?produced_by_uri <http://www.w3.org/2004/02/skos/core#prefLabel> ?produced_by_label .
    FILTER (lang(?produced_by_label) = "en")

    VALUES ?product_uri {%s}
} LIMIT 1000
"""%("<" + "> <".join(agrovoc_wrong_germplasms.uri) + ">")

In [12]:
sparql.setQuery(query)
ret = sparql.queryAndConvert()
tmp_d = {"product_uri" : [], "product_label" : [], "produced_by_uri" : [], "produced_by_label" : []}
for row in ret["results"]["bindings"]:
    tmp_d["product_uri"].append(row["product_uri"]["value"])
    tmp_d["product_label"].append(row["product_label"]["value"])
    tmp_d["produced_by_uri"].append(row["produced_by_uri"]["value"])
    tmp_d["produced_by_label"].append(row["produced_by_label"]["value"])
agrovoc_produces = pd.DataFrame(tmp_d)
agrovoc_produces

,product_uri,product_label,produced_by_uri,produced_by_label
0,http://aims.fao.org/aos/agrovoc/c_12332,maize,http://aims.fao.org/aos/agrovoc/c_8504,Zea mays
1,http://aims.fao.org/aos/agrovoc/c_14386,soft wheat,http://aims.fao.org/aos/agrovoc/c_7951,Triticum aestivum
2,http://aims.fao.org/aos/agrovoc/c_11456,hard wheat,http://aims.fao.org/aos/agrovoc/c_7955,Triticum durum
3,http://aims.fao.org/aos/agrovoc/c_7499,sugar beet,http://aims.fao.org/aos/agrovoc/c_890,Beta vulgaris
4,http://aims.fao.org/aos/agrovoc/c_3558,hemp,http://aims.fao.org/aos/agrovoc/c_1257,Cannabis sativa
5,http://aims.fao.org/aos/agrovoc/c_14477,soybeans,http://aims.fao.org/aos/agrovoc/c_3301,Glycine max
6,http://aims.fao.org/aos/agrovoc/c_14477,soybeans,http://aims.fao.org/aos/agrovoc/c_23607,Glycine soja


In [13]:
agrovoc_wrong_germplasms[~agrovoc_wrong_germplasms.uri.isin(agrovoc_produces.product_uri)].to_csv("agrovoc_probable_mismatches.csv", index=False)
agrovoc_produces.to_csv("agrovoc_produces_mismatches.csv", index=False)

---      

# <br>        